In [79]:
import requests
import pandas as pd
import mysql.connector
import config
# from mysql.connector import errorcode
# import json

In [80]:
# Define function that makes a call to fbi api and returns specified data in a list format
def fbi_call(state):
    url = f'https://api.usa.gov/crime/fbi/sapi/api/summarized/estimates/states/{state}/1979/2018?API_KEY={config.fbi_api_key}'
    response = requests.get(url)
    data = response.json()
    crime_data = []
    for entry in data['results']:
        entry_tuple = [(entry['state_abbr'] + str(entry['year'])), entry['state_abbr'], entry['year'], entry['population'], entry['violent_crime']]
        crime_data.append(entry_tuple)
    crime_data.sort(key=lambda x: x[2])
    return crime_data

In [81]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [82]:
# Getting the data for each state by iterating through a list of states and calling the fbi_call function on each
all_states_crime_data = []
for state in states:
    all_states_crime_data.extend(fbi_call(state))

In [83]:
url = f'https://api.usa.gov/crime/fbi/sapi/api/summarized/estimates/states/{state}/1979/2018?API_KEY={config.fbi_api_key}'
response = requests.get(url)
data = response.json()
list_keys = data['results'][0].keys()
for i, value in enumerate(list_keys):
    print(f'{i} - {value}')

0 - state_id
1 - state_abbr
2 - year
3 - population
4 - violent_crime
5 - homicide
6 - rape_legacy
7 - rape_revised
8 - robbery
9 - aggravated_assault
10 - property_crime
11 - burglary
12 - larceny
13 - motor_vehicle_theft
14 - arson


In [71]:
# def fbi_call(state):
#     url = f'https://api.usa.gov/crime/fbi/sapi/api/summarized/estimates/states/{state}/1979/2018?API_KEY={config.fbi_api_key}'
#     response = requests.get(url)
#     data = response.json()
#     populatio
#     for entry in data['results']:
        
#         entry_tuple = [(entry['state_abbr'] + str(entry['year']), entry['state_abbr'], entry['year'], entry['population'], entry['violent_crime']
#         crime_data.append(entry_tuple)
#     crime_data.sort(key=lambda x: x[2])
#     return crime_data

In [72]:
all_states_crime_data

[['AL1979', 'AL', 1979, 3769000, 15578],
 ['AL1980', 'AL', 1980, 3861466, 17320],
 ['AL1981', 'AL', 1981, 3916000, 18423],
 ['AL1982', 'AL', 1982, 3943000, 17653],
 ['AL1983', 'AL', 1983, 3959000, 16471],
 ['AL1984', 'AL', 1984, 3990000, 17204],
 ['AL1985', 'AL', 1985, 4021000, 18398],
 ['AL1986', 'AL', 1986, 4053000, 22616],
 ['AL1987', 'AL', 1987, 4083000, 22833],
 ['AL1988', 'AL', 1988, 4127000, 23052],
 ['AL1989', 'AL', 1989, 4118000, 24329],
 ['AL1990', 'AL', 1990, 4040587, 28630],
 ['AL1991', 'AL', 1991, 4089000, 34518],
 ['AL1992', 'AL', 1992, 4136000, 36052],
 ['AL1993', 'AL', 1993, 4187000, 32676],
 ['AL1994', 'AL', 1994, 4219000, 28844],
 ['AL1995', 'AL', 1995, 4253000, 26894],
 ['AL1996', 'AL', 1996, 4273000, 24159],
 ['AL1997', 'AL', 1997, 4319000, 24379],
 ['AL1998', 'AL', 1998, 4352000, 22286],
 ['AL1999', 'AL', 1999, 4369862, 21421],
 ['AL2000', 'AL', 2000, 4447100, 21620],
 ['AL2001', 'AL', 2001, 4468912, 19582],
 ['AL2002', 'AL', 2002, 4478896, 19931],
 ['AL2003', 'AL'

Now that we have our data in the format of a list of tuples we will add it to our database in sql

In [73]:
db_name = 'mod3_eda'

In [74]:
# Create a connection
cnx = mysql.connector.connect(
    host = config.host,
    user = 'admin',
    passwd = config.password,
    database = db_name
)

In [75]:
# Create / close cursor
cursor = cnx.cursor()
# cursor.close()

In [76]:
# Create table description
table_description = ("""
CREATE TABLE IF NOT EXISTS fbi_crime_data (
    year_state_key VARCHAR(255) NOT NULL PRIMARY KEY,
    state VARCHAR(255) NOT NULL,
    year INT NOT NULL,
    population INT NOT NULL,
    violent_crimes_count INT NOT NULL
)  ENGINE=INNODB;
""")

In [77]:
# define insert query
fbi_table_query = """INSERT INTO fbi_crime_data
    (year_state_key, state, year, population, violent_crimes_count)
    VALUES (%s, %s, %s, %s, %s)"""

In [63]:
# Function to insert results into a table based on the given query
def db_insert(data, table_qry):
    for n in range(0, len(data)):
        cursor.execute(table_qry, data[n])
        print(f'inserted a row for {n} row in results')

In [64]:
# Create table 
cursor.execute(table_description)

In [78]:
# Call db_insert function to insert data into table
db_insert(all_states_crime_data, fbi_table_query)

inserted a row for 0 row in results
inserted a row for 1 row in results
inserted a row for 2 row in results
inserted a row for 3 row in results
inserted a row for 4 row in results
inserted a row for 5 row in results
inserted a row for 6 row in results
inserted a row for 7 row in results
inserted a row for 8 row in results
inserted a row for 9 row in results
inserted a row for 10 row in results
inserted a row for 11 row in results
inserted a row for 12 row in results
inserted a row for 13 row in results
inserted a row for 14 row in results
inserted a row for 15 row in results
inserted a row for 16 row in results
inserted a row for 17 row in results
inserted a row for 18 row in results
inserted a row for 19 row in results
inserted a row for 20 row in results
inserted a row for 21 row in results
inserted a row for 22 row in results
inserted a row for 23 row in results
inserted a row for 24 row in results
inserted a row for 25 row in results
inserted a row for 26 row in results
inserted a 

inserted a row for 225 row in results
inserted a row for 226 row in results
inserted a row for 227 row in results
inserted a row for 228 row in results
inserted a row for 229 row in results
inserted a row for 230 row in results
inserted a row for 231 row in results
inserted a row for 232 row in results
inserted a row for 233 row in results
inserted a row for 234 row in results
inserted a row for 235 row in results
inserted a row for 236 row in results
inserted a row for 237 row in results
inserted a row for 238 row in results
inserted a row for 239 row in results
inserted a row for 240 row in results
inserted a row for 241 row in results
inserted a row for 242 row in results
inserted a row for 243 row in results
inserted a row for 244 row in results
inserted a row for 245 row in results
inserted a row for 246 row in results
inserted a row for 247 row in results
inserted a row for 248 row in results
inserted a row for 249 row in results
inserted a row for 250 row in results
inserted a r

inserted a row for 441 row in results
inserted a row for 442 row in results
inserted a row for 443 row in results
inserted a row for 444 row in results
inserted a row for 445 row in results
inserted a row for 446 row in results
inserted a row for 447 row in results
inserted a row for 448 row in results
inserted a row for 449 row in results
inserted a row for 450 row in results
inserted a row for 451 row in results
inserted a row for 452 row in results
inserted a row for 453 row in results
inserted a row for 454 row in results
inserted a row for 455 row in results
inserted a row for 456 row in results
inserted a row for 457 row in results
inserted a row for 458 row in results
inserted a row for 459 row in results
inserted a row for 460 row in results
inserted a row for 461 row in results
inserted a row for 462 row in results
inserted a row for 463 row in results
inserted a row for 464 row in results
inserted a row for 465 row in results
inserted a row for 466 row in results
inserted a r

inserted a row for 659 row in results
inserted a row for 660 row in results
inserted a row for 661 row in results
inserted a row for 662 row in results
inserted a row for 663 row in results
inserted a row for 664 row in results
inserted a row for 665 row in results
inserted a row for 666 row in results
inserted a row for 667 row in results
inserted a row for 668 row in results
inserted a row for 669 row in results
inserted a row for 670 row in results
inserted a row for 671 row in results
inserted a row for 672 row in results
inserted a row for 673 row in results
inserted a row for 674 row in results
inserted a row for 675 row in results
inserted a row for 676 row in results
inserted a row for 677 row in results
inserted a row for 678 row in results
inserted a row for 679 row in results
inserted a row for 680 row in results
inserted a row for 681 row in results
inserted a row for 682 row in results
inserted a row for 683 row in results
inserted a row for 684 row in results
inserted a r

inserted a row for 876 row in results
inserted a row for 877 row in results
inserted a row for 878 row in results
inserted a row for 879 row in results
inserted a row for 880 row in results


IntegrityError: 1062 (23000): Duplicate entry 'MI1979' for key 'PRIMARY'

In [ ]:
# Commit the changes to database
cnx.commit()

In [ ]:
cursor.close()

In [50]:
cursor.execute('SELECT * FROM mod3_eda.fbi_crime_data').fetchall()

AttributeError: 'NoneType' object has no attribute 'fetchall'

In [49]:
cursor.fetchall()

InterfaceError: No result set to fetch from.